# Quasi-criticality in the Cortex

In [ ]:
'''Setup notebook environment -q flag suppresses output, if you want to see it, remove the -q flag'''
%pip install -r requirements.txt -q
from utils.plotting_utils import power_law_plot
from utils.data_utils import load_data

In [ ]:
data = load_data('data/avalanches.txt')

In [ ]:
power_law_plot(data)